### 단순한 계층 구현하기
계층 : 신경망의 기능 단위



In [ ]:
import numpy as np

곱셈계층

In [ ]:

class MulLayer:
  def __init__(self):
    self.x = None
    self.y = None

  def forward(self,x,y):  # 순전파
    self.x = x
    self.y = y
    out = x * y

    return out

  def backward(self, dout):  # 역전파
    dx = dout * self.y  # x와 y를 바꾼다
    dy = dout * self.x

    return dx, dy


In [ ]:
apple = 100
apple_num = 2
tax = 1.1

#계층들
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

#순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print(price) # 220

# 역전파
dprice = 1
dapple_price,dtax = mul_tax_layer.backward(dprice)
dapple,dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dtax) # 2.2 110 200

### 뎃셈계층

In [ ]:
class AddLayer:
  def __init__(self):
    pass  # 덧셈계층에서는 초기화 필요x

  def forward(self, x, y):
    out = x + y
    return out

  def backward(self, dout):
    dx = dout * 1
    dy = dout * 1
    return dx, dy

In [ ]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# 계층들
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# 순전파

apple_price = mul_apple_layer.fowrard(apple,apple_num)
orange_price = mul_orange_layer.forward(orange, orange_num)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(all_price, tax)

# 역전파
dprice = 1
dall_price,dtax = mul_tax_layer.backward(dprice)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

price(price)
print(dapple_num, dapple, dorange, dorange_num, dtax)

###  활성화 함수 계층 구현하기

ReLU 계층

In [ ]:
class Relu:
  def __init__(self):
    self.mask = None
  
  def forward(self,x):
    self.mask = (x <= 0)  # mask는 True/False로 구성된 넘파이 배열. x <=0 이면 True, x>0이면 False
    out = x.copy()
    out[self.maxk] = 0

    return out

  def backward(self, dout):
    dout[self.mask] = 0
    dx = dout
  
    return dx

sigmoid계층

In [ ]:
class Sigmoid:
  def __init__(self):
    self.out = None

  def forward(self,x):
    out = 1 / (1+ np.exp(-x))
    self.out = out
  def backward(self, dout):
    dx = dout * (1.0 - self.out) * self.out

    return dx

### Affine/Softmax 계층 구현하기

Affine계층

배치용 Affine계층

In [ ]:
class Affine:
  def __init__(self, w, b):
    self.w = w
    self.b = b
    self.x = None
    self.dw = None
    self.db = None

  def forward(self, x):
    self.x = x
    out = np.dot(x,self.w) + self.b

  def backward(self, dout):
    dx = np.dot(dout,self.w.t)
    self.dw = np.dot(self.x.t, dout)
    self.db = np.sum(dout, axis = 0)

    return dx

### Softmax - with - Loss 계층

In [ ]:
# 개선된 softmax함수
def softmax(a):
  c = np.max(a)
  exp_a = np.exp(a-c)  # 오버플로 대책
  sum_exp_a = np.sum(exp_a)
  y = exp_a / sum_exp_a

  return y

In [ ]:
#(배치용) 교차 엔트로피 오차 구현(정답 레이블 t가 one-hot-encoding일 때)
def cross_entropy_error(y,t):  # y: 신경망의 출력, t : 정답 레이블
  if y.ndim == 1:
    t = t.reshape(1,t.size)  # reshape : 데이터의 형상을 바꿈
    y = y.reshape(1,y.size)
  batch_size = y.shape[0]
  return -np.sum(t*np.log(y+1e-7)) / batch_size  # 배치의 크기로 나누어 정규화한 후 1장당 평균의 교차 엔트로피 오차를 계산

In [ ]:
class SoftmaxWithLoss:
  def __init__ (self):
    self.loss = None  # 손실
    self.y = None  # softmax의 출력
    self.t = None  # 정답 레이블(원-핫 벡터)

  def forward(self, x, t):
    self.t = t
    self.y = softmax(x)
    self.loss = cross_entropy_error(self,y,self.t)
    return self.loss
  def backward(self,dout = 1):
    batch_size = self.t.shape[0]
    ds = (self.y - self.t) / batch_size

### 오차역전파법 구현하기

## 신경망의 전체 그림
전제) 신경망에는 적응 가능한 가중치와 편향이 있고, 이 가중치와 편향을 훈련데이터에 적응하도록 조정하는 과정을 '학습'이라고 한다.

1단계 - 미니배치
훈련 데이터 중 일부를 무작위로 가져온다. > 미니배치
목표 : 미니배치의 손실함수값을 줄이는 것

2단게 - 기울기 산출
미니배치함수의 손실함수 값을 줄이기 위해 각 가중치 매개변수의 기울기를 구한다.
기울기는 손실함수의 값을 가장 작게 하는 방향을 제시한다.

3단계 - 매개변수 갱신 
가중치 매개변수를 기울기 방향으로 아주 조금 갱신한다.

4단계 - 반복
1~3단계를 반복


In [ ]:
import os , sys
sys.path.append()
import numpy as np
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict

class TwoLayerNet:
  def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
    # 가중치 초기화
    self.params = {}
    self.params = ['w1'] = weight_init_std * np.random.radn(input_size, hidden_size)
    self.params = ['b1'] = np.zeros(hidden_size)
    self.params = ['w2'] = weight_init_std * np.randmo.randn(hidden_size, output_size)
    self.params = ['b2'] = np.zeros(output_size)

    # 계층 생성
    self.layers = OrderedDict()
    self.layers['Affine1'] = Affine(self.params['w1'], self.params['b1'])
    self.layers['Relu1'] = Relu()
    self.layers['Affine2'] = Affine(self.params['w2'],self.params['b2'])

    self.lastLayer = SoftmaxWithLoss()

  def predict(self,x):
    for layer in self.layers.values():
      x = layer.forward()
    return x

  # x : 입력 데이터, t : 정답 레이블

  def loss(self, x, t):
    y = self.predict(x)
    return self.lastLayer.forward(y,t)

  def accuracy(self,x,t):
    y = self.predict(x)
    y = np.argmax(y, axis = 1)
    if t.ndim != 1:
      t = np.argmax(t, axis = 1)

      accuracy = np.sum(y == t) / float(x.shape[0])
    return accuracy


    # x: 입력 데이터, t: 정답 레이블
    def numerical_gradient(self, x, t):
      loss_w = lambda w:self.loss(x,t)

      grads = {}
      grads['w1'] = numerical_gradient(loss_w, self.params['w1'])
      grads['b1'] = numerical_gradient(loss_w, self.params['b1'])
      grads['w2'] = numerical_gradient(loss_w, self.params['w2'])
      grads['b2'] = numerical_gradient(loss_w, self.params['b2'])
      return grads

    def gradient(self,x,t):
      # 순전파
      self.loss(x,t)

      # 역전파
      dout = 1
      dout = self.lastLayer.backward(dout)

      layers = list(self.layers.valuees())
      laysers.reverse()
      for layer in layers:
        dout = layer.backward(dout)

      # 결과 저장
      grads = {}
      grads['w1'] = self.layers['Affine1'].dw
      grads['b1'] = self.layers['Affine1'].db
      grads['w2'] = self.layers['Affine2'].dw
      grads['b2'] = self.layers['Affine2'].db

      return grads

### 오차 역전파법으로 구한 기울기 검증하기

In [ ]:
import sys, os
sys.path.append()
impor numpy as np
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet

# 데이터 읽기
(x_train, t_train),(x_test, t_test) = load_mnist(normalize = True, one_hot_label = True)

network = TwoLayerNet(input_size = 784, hidden_size = 50, output_size = 10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

# 각 가중치의 차이의 절댓값을 구한후, 그 절댓값들의 평균을 낸다.
for key in grad_numerical.keys():
  diff = np.average(np.abs(grad_backprop[key] - grad_numerical[key]))
  print(key + "." + str(diff))
  

### 오차역전파법을 사용한 학습 구현하기

In [ ]:
import sys, os
sys.path.append()
import numpy as np
from dataset.mnist imoprt load_mnist
from two_layer_net import TwoLayerNet

# 데이터 읽기
(x_train,t_train),(x_test, t_test) = load_mnist(normalize = True, one_hot_label = True)
network = TwoLayerNet(input_size = 784, hidden_size = 50, output_size = 10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
  batch_mask = np.random.choice(train_size, batch_size)
  x_batch = x_train[batch_mask]
  t_batch = t_train[batch_mask]

  # 오차역전파법으로 기울기를 구한다.
  grad = network.gradient(x_batch, t_batch)

  #갱신
  for key in('w1','b1','w2','b2'):
    network.params[key] -= learning_rate * grad[key]

  loss = network.loss(x_batch, t_batch)
  train_loss_list.append(loss)

  if i % iter_per_epoch == 0:
    train_acc = network.accuracy(x_train, t_train)
    test_acc = network.accuracy(x_test, t_test)
    train_acc_list.append(train_acc)
    test_acc_list.append(test_acc)
    print(train_acc, test_acc)